In [1]:
from vivit import run_experiment, test_files, classes
from data import get_video_label, VideoDataGenerator, labels_df, files, labels
import synthetic
import scipy.stats as stats
from sklearn.model_selection import train_test_split
import numpy as np
import keras
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [2]:
classes = list(labels_df.groupby('label').size().sort_values(ascending=False)[:10].index)
good_classes = ['book', "drink", "go", "chair", "who", "clothes", "yes", "year", "thin", "fine"]
test_files = [f for f in files if labels[int(os.path.basename(f).split('.')[0])] in good_classes]

In [3]:
base = "../data/videos"
for video, f in enumerate(test_files):
    name = os.path.basename(f)
    if not os.path.exists(f"{base}/flipped/{name}"):
        synthetic.flip_video(f"{base}/original/{name}", f"{base}/flipped/{name}")
    if not os.path.exists(f"{base}/rotated_90/{name}"):
        synthetic.rotate_video(f"{base}/original/{name}", f"{base}/rotated_90/{name}", 90)
    if not os.path.exists(f"{base}/rotated_270/{name}"):
        synthetic.rotate_video(f"{base}/original/{name}", f"{base}/rotated_270/{name}", 270)
    if not os.path.exists(f"{base}/noised/{name}"):
        synthetic.noise_video(f"{base}/original/{name}", f"{base}/noised/{name}")
    print(f"{video+1}/{len(test_files)}", end = '\r')

In [4]:
all_files = []
for folder in ["original", "flipped", "rotated_90", "rotated_270", "noised"]:
    all_files += [f"{base}/{folder}/{os.path.basename(f)}" for f in test_files]
y = [get_video_label(f) for f in all_files]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(all_files, y, test_size=0.2, random_state=42, stratify=y)
train, validation = VideoDataGenerator(X_train), VideoDataGenerator(X_test)

In [8]:
model = run_experiment(name = "decay", data = train, validation = validation, epochs = 100)

Epoch 1/100


2024-04-16 13:44:06.800563: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


34/34 ━━━━━━━━━━━━━━━━━━━━ 117s 3s/step - accuracy: 0.0741 - loss: 6.7720 - top-5-accuracy: 0.3072 - val_accuracy: 0.1585 - val_loss: 5.5919 - val_top-5-accuracy: 0.5736
Epoch 2/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 73s 2s/step - accuracy: 0.1900 - loss: 5.3695 - top-5-accuracy: 0.6371 - val_accuracy: 0.1396 - val_loss: 4.6738 - val_top-5-accuracy: 0.6113
Epoch 3/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 62s 2s/step - accuracy: 0.1571 - loss: 4.4745 - top-5-accuracy: 0.5946 - val_accuracy: 0.1585 - val_loss: 3.8156 - val_top-5-accuracy: 0.5925
Epoch 4/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 60s 2s/step - accuracy: 0.1680 - loss: 3.6573 - top-5-accuracy: 0.6309 - val_accuracy: 0.1509 - val_loss: 3.1688 - val_top-5-accuracy: 0.6189
Epoch 5/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 62s 2s/step - accuracy: 0.1391 - loss: 3.0861 - top-5-accuracy: 0.6062 - val_accuracy: 0.1585 - val_loss: 2.7754 - val_top-5-accuracy: 0.6226
Epoch 6/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.1585 - loss: 2.7730 - top-5-accuracy: 0.5

In [9]:
model.save("../models/END-decay.keras")